# 07 Semantic Kernel | 01 Planner

## Azure Environment

To execute the sample code Azure service specific information like endpoint, api key etc. is needed ([Details and instructions can be found here](../01_CreateEnvironment/01_Environment.ipynb))

This notebooks highlights how Semantic Kernel simplifies the process of storing and retrieving vector data. 

## Step 1: Create Semantic Kernel

MS Semantic Kernel is a tool or library developed by Microsoft to abstract away communication with LLMs and other services like vector DBs. The nuget package Microsoft.SemanticKernel provides developers with access to this powerful tool, allowing them to integrate advanced language processing capabilities into their applications effortlessly.


In [6]:
#r "nuget: Microsoft.SemanticKernel, 1.15.1"
#r "nuget: Microsoft.SemanticKernel.Connectors.OpenAI, 1.15.1"
#r "nuget: DotNetEnv, 2.5.0"

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using DotNetEnv;

//configuration file is created during environment creation
static string _configurationFile = @"../../Configuration/application.env";
Env.Load(_configurationFile);

string oAiApiKey = Environment.GetEnvironmentVariable("WS_AOAI_APIKEY") ?? "WS_AOAI_APIKEY not found";
string oAiEndpoint = Environment.GetEnvironmentVariable("WS_AOAI_ENDPOINT") ?? "WS_AOAI_ENDPOINT not found";
string chatCompletionDeploymentName = Environment.GetEnvironmentVariable("WS_CHATCOMPLETION_DEPLOYMENTNAME") ?? "WS_CHATCOMPLETION_DEPLOYMENTNAME not found";
string embeddingDeploymentName = Environment.GetEnvironmentVariable("WS_EMBEDDING_DEPLOYMENTNAME") ?? "WS_EMBEDDING_DEPLOYMENTNAME not found";
string assetFolder = Environment.GetEnvironmentVariable("WS_ASSETS_FOLDER") ?? "WS_ASSETS_FOLDER not found";  


var kernel = Kernel
    .CreateBuilder()
    .AddAzureOpenAIChatCompletion(
        chatCompletionDeploymentName,
        oAiEndpoint,
        oAiApiKey
    )
    .Build();
    
Console.WriteLine($"Semantic Kernel created...");


Installed Packages DotNetEnv, 2.5.0 Microsoft.SemanticKernel, 1.15.1 Microsoft.SemanticKernel.Connectors.OpenAI, 1.15.1

Semantic Kernel created...


## Step 3: Add Plugins to the planner


In [7]:
private string GetSportEventWinner(string sportEvent, string year)
{
    // Implement the logic to get the winner of the sport event.
    Console.WriteLine("GetSportEventWinner called...");
    return "Kansas City Chiefs";
}

private string GetSportEventResult(string sportEvent, string year)
{
    // Implement the logic to get the result of the sport event.
    Console.WriteLine("GetSportEventResult called...");
    return "25:22";
}

KernelFunction kernelFunctionWinner = kernel.CreateFunctionFromMethod(
    GetSportEventWinner, 
    "GetSportEventWinner", 
    "Get the winner of the sport event."
);
KernelFunction kernelFunctionResult = kernel.CreateFunctionFromMethod(
    GetSportEventResult, 
    "GetSportEventResult", 
    "Get the result of the sport event."
);

kernel.ImportPluginFromFunctions(
    "SportFunctions", 
    new KernelFunction[] { 
        kernelFunctionWinner, 
        kernelFunctionResult 
    }
);

Console.WriteLine($"Sport functions imported...");

Sport functions imported...


## Step 4: Call Functions

In [8]:
IChatCompletionService chatCompletionService = kernel.GetRequiredService<IChatCompletionService>(); 

// Allow Auto Invoke of provided plugins
OpenAIPromptExecutionSettings openAIPromptExecutionSettings = new() 
{
    ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions
};

ChatHistory chatHistory = new ChatHistory();
chatHistory.AddUserMessage("Who won the Super Bowl 2024? What was the result?");

ChatMessageContent chatMessageContent = await chatCompletionService.GetChatMessageContentAsync(
    chatHistory,
    executionSettings: openAIPromptExecutionSettings,
    kernel: kernel);

Console.WriteLine($"Execution results: {chatMessageContent}");

GetSportEventWinner called...
GetSportEventResult called...
Execution results: The winner of the Super Bowl 2024 was the Kansas City Chiefs. The result was 25-22.
